In [406]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing
import dateutil.parser
#yourdate = dateutil.parser.parse(datestring)
from time import gmtime, strftime


In [407]:
needed = ["NAME", "MON/WED/FRI (FRIDAYS UNTIL 5:00 PM)", "TUE/THU", "SATURDAY", "SUNDAY"]
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
#If needed declarations

In [408]:
#ref is column entry, col is column name
def check(ref, col):
    day = [""]
    time = ""
    #MWF
    if col == needed[1]:
        day = [days[0], days[2], days[4]]
    #TT
    elif col == needed[2]:
        day = [days[1], days[3]]
    #SAT
    elif col == needed[3]:
        day = [days[5]]
    #SUN
    elif col == needed[4]:
        day = [days[6]]

    for i in range(0, len(day)):
        #Check if time is empty to avoid comma at the start and build the string
        if not time:
            time = day[i] + ", " +duration(day[i], ref)
        else:
            time = time +"; "+ day[i] + "," +duration(day[i], ref)
    return time

    #return day

In [409]:
def duration(day, reference):
    starttime = stoptime = 'null' 
    #print("Entered duration")
    if day.lower() == "Sunday".lower() or day.lower() == "Satuday".lower():
            starttime = stoptime = dateutil.parser.parse("12:00AM").time()  
            
            if reference.find("Afternoon") != -1:
                starttime = dateutil.parser.parse("1PM").time()
                if day.lower() == "Sunday".lower():
                    stoptime = dateutil.parser.parse("6PM") .time() 
                else:
                    stoptime = dateutil.parser.parse("5PM") .time()
                    
            elif reference.find("Closing") != -1:
                starttime = dateutil.parser.parse("6PM").time()
                stoptime = dateutil.parser.parse("10PM") .time() 
                
    else:
        starttime = stoptime = dateutil.parser.parse("12:00AM").time()
        #print("Not Sunday or Saturday")
        if reference.find("Opening") != -1:
            #print("Entered Opening")
            starttime = dateutil.parser.parse("7:30AM").time()
            stoptime = dateutil.parser.parse("10AM") .time()   
             
        elif reference.find("Morning") != -1:
            #print("Entered Morning")  
            starttime = dateutil.parser.parse("10AM").time()
            stoptime = dateutil.parser.parse("1PM") .time() 
         
        elif reference.find("Mid Day") != -1:
            #print("Entered Mid Day")  
            starttime = dateutil.parser.parse("1PM").time()
            if day.lower() == "Friday".lower():
                stoptime = dateutil.parser.parse("5PM") .time() 
            else:
                stoptime = dateutil.parser.parse("4PM") .time() 
                
        elif reference.find("Afternoon") != -1:
            starttime = dateutil.parser.parse("4PM").time()
            stoptime = dateutil.parser.parse("7PM") .time() 
            
        elif reference.find("Closing") != -1:
            starttime = dateutil.parser.parse("7PM").time()
            stoptime = dateutil.parser.parse("10PM") .time() 
                
    timeframe = starttime.strftime("%I:%M %p") + " - " + stoptime.strftime("%I:%M %p")            
    return timeframe

In [410]:
duration('Friday', 'Afternoon')

'04:00 PM - 07:00 PM'

In [411]:
#Testing Check function
check('Mid Day (1pm-4pm) (Fridays 1pm-5pm)','MON/WED/FRI (FRIDAYS UNTIL 5:00 PM)')

'Monday, 01:00 PM - 04:00 PM; Wednesday,01:00 PM - 04:00 PM; Friday,01:00 PM - 05:00 PM'

In [412]:
check('Morning (10am-1pm)', 'TUE/THU')

'Tuesday, 10:00 AM - 01:00 PM; Thursday,10:00 AM - 01:00 PM'

In [413]:
data = pd.read_csv("/Users/Rohit/Desktop/schedule.csv")
data.columns = map(str.upper, data.columns)

# Data Management
list(data.columns.values)

['TIMESTAMP',
 'USERNAME',
 'NAME',
 'HOW MANY HOURS WOULD YOU LIKE TO WORK DURING THE WEEK?',
 'MONDAY',
 'TUESDAY',
 'WEDNESDAY',
 'THURSDAY',
 'FRIDAY',
 'DO YOU HAVE ANY ADDITIONAL ACADEMIC OBLIGATIONS, I.E. LABS, CLUBS, SPORTS...? (PLEASE PROVIDE SPECIFIC DATES/TIMES)',
 'MON/WED/FRI (FRIDAYS UNTIL 5:00 PM)',
 'TUE/THU',
 'SATURDAY',
 'SUNDAY',
 'ADDITIONAL COMMENTS/REQUESTS FOR YOUR FALL WORK SCHEDULE?']

In [414]:
clean_data = data[needed] #Necessary Columns
clean_data

,NAME,MON/WED/FRI (FRIDAYS UNTIL 5:00 PM),TUE/THU,SATURDAY,SUNDAY
0,Davis Allen,Opening (7:30am-10am),"Opening (7:30am-10am), Morning (10am-1pm)",NaN,NaN
1,Dante Dalla Gasperina,"Opening (7:30am-10am), Mid Day (1pm-4pm) (Frid...",Mid Day (1pm-4pm),NaN,NaN
2,JT Tran,"Morning (10am-1pm), Mid Day (1pm-4pm) (Fridays...",NaN,NaN,NaN
3,Grecia Escobar,"Morning (10am-1pm), Mid Day (1pm-4pm) (Fridays...","Opening (7:30am-10am), Morning (10am-1pm)",NaN,NaN
4,Stewart Allen,"Afternoon (4pm-7pm), Closing (7pm-10pm)","Opening (7:30am-10am), Mid Day (1pm-4pm), Afte...",Afternoon (1pm-5pm),Afternoon (1pm-6pm)
5,Francesco Petrini,"Mid Day (1pm-4pm) (Fridays 1pm-5pm), Afternoon...","Mid Day (1pm-4pm), Afternoon (4pm-7pm), Closin...",NaN,NaN
6,Jessica Battad,"Opening (7:30am-10am), Morning (10am-1pm)",Opening (7:30am-10am),NaN,NaN
7,Deion Graham-Long,Mid Day (1pm-4pm) (Fridays 1pm-5pm),"Mid Day (1pm-4pm), Afternoon (4pm-7pm)",Afternoon (1pm-5pm),Afternoon (1pm-6pm)
8,Pranav Bheda,"Morning (10am-1pm), Afternoon (4pm-7pm)","Morning (10am-1pm), Afternoon (4pm-7pm), Closi...",NaN,NaN
9,Brendan Appelman,Afternoon (4pm-7pm),Closing (7pm-10pm),NaN,NaN


In [415]:
f = open("/Users/Rohit/Desktop/test.txt",'w')

In [416]:
# Table (Name, Availability) columns for each section
for i in range(1, clean_data.shape[1]):
    temp = clean_data[pd.notnull(clean_data[needed[i]])]
    
    #Needed data at a time
    col = [needed[0], needed[i]]
    temp = temp[col] 
    
    #Splitting shifts initially from csv dataframe import
    s = temp[needed[i]].str.split(',').apply(Series, 1).stack()
    s.index = s.index.droplevel(-1)
    s.name = needed[i]
    del temp[needed[i]]
    temp = temp.join(s).reset_index(drop=True)
    
    #Applying check() to convert to readable form for every row (shape[0])
    for j in range(0, temp.shape[0]): 
        #Cleaning the 2nd column
        temp.at[j, needed[i]] =  check(temp.at[j, needed[i]], needed[i]) #check(reference, day)
        #Splitting single availability column to day-by-day columns
        new = pd.DataFrame(temp[needed[i]].apply(lambda x: x.split('; ')).values.tolist())
        #Merge the day-by-day with the name column to get the desired result
        result = pd.concat([temp[needed[0]], new], axis=1)
        
        
    #Test Case
    if(i == 1):
        mwf = result
    elif(i == 2):
        tt = result
    elif(i == 3):
        sat = result
    elif(i == 4):
        sun = result
    
    #temp.to_csv(r"/Users/Rohit/Desktop/test.txt", header='\n', index=None, mode='a', sep='\n') 
#list(sat.columns.values)[1]

In [417]:
#Monday
mon = mwf[[needed[0], 0]]
#Wednesday
wed = mwf[[needed[0], 1]]
#Friday
fri = mwf[[needed[0], 2]]
#Tuesday
tue = tt[[needed[0], 0]]
#Thursday
thu = tt[[needed[0], 1]]

week = [mon, tue, wed, thu, fri, sat, sun]

#df['MEETING START TIME'] = df['MEETING START TIME'].map(lambda x: pd.datetools.parse(x))

for i in range(0, len(wed)):
    print(wed[wed.columns[1]].str.split(',')[i][1])

07:30 AM - 10:00 AM
07:30 AM - 10:00 AM
01:00 PM - 04:00 PM
10:00 AM - 01:00 PM
01:00 PM - 04:00 PM
10:00 AM - 01:00 PM
01:00 PM - 04:00 PM
04:00 PM - 07:00 PM
07:00 PM - 10:00 PM
01:00 PM - 04:00 PM
04:00 PM - 07:00 PM
07:30 AM - 10:00 AM
10:00 AM - 01:00 PM
01:00 PM - 04:00 PM
10:00 AM - 01:00 PM
04:00 PM - 07:00 PM
04:00 PM - 07:00 PM
10:00 AM - 01:00 PM
01:00 PM - 04:00 PM
07:00 PM - 10:00 PM
07:00 PM - 10:00 PM
10:00 AM - 01:00 PM
07:00 PM - 10:00 PM
10:00 AM - 01:00 PM
01:00 PM - 04:00 PM
01:00 PM - 04:00 PM
07:00 PM - 10:00 PM
07:30 AM - 10:00 AM
01:00 PM - 04:00 PM
01:00 PM - 04:00 PM
04:00 PM - 07:00 PM


In [424]:
for day in week:
    for i in range(0, len(day)):
        #day[day.columns[1]].str.replace(day[day.columns[1]], day[day.columns[1]].str.split(',')[i][1])
        #WORKS! day[day.columns[1]] = day[day.columns[1]].apply(lambda x: pd.Series(x.split(',')))
        day.loc[i,day.column]
    print(day)

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


                     NAME       0
0             Davis Allen  Monday
1   Dante Dalla Gasperina  Monday
2   Dante Dalla Gasperina  Monday
3                 JT Tran  Monday
4                 JT Tran  Monday
5          Grecia Escobar  Monday
6          Grecia Escobar  Monday
7           Stewart Allen  Monday
8           Stewart Allen  Monday
9       Francesco Petrini  Monday
10      Francesco Petrini  Monday
11         Jessica Battad  Monday
12         Jessica Battad  Monday
13      Deion Graham-Long  Monday
14           Pranav Bheda  Monday
15           Pranav Bheda  Monday
16       Brendan Appelman  Monday
17         Andrew Schmidt  Monday
18         Andrew Schmidt  Monday
19        Natalie Neumann  Monday
20        Tasmine Hackson  Monday
21          Robert Scalzo  Monday
22          Robert Scalzo  Monday
23     Sanjay Tamizharasu  Monday
24     Sanjay Tamizharasu  Monday
25          Daniel Jensen  Monday
26        Dominic Komarek  Monday
27           Parker Gribb  Monday
28           P

In [420]:
temp

,0,1
0,Sunday,01:00 PM - 06:00 PM
1,Sunday,01:00 PM - 06:00 PM
2,Sunday,01:00 PM - 06:00 PM
3,Sunday,06:00 PM - 10:00 PM
4,Sunday,06:00 PM - 10:00 PM
5,Sunday,01:00 PM - 06:00 PM
6,Sunday,06:00 PM - 10:00 PM
7,Sunday,01:00 PM - 06:00 PM
8,Sunday,06:00 PM - 10:00 PM
9,Sunday,06:00 PM - 10:00 PM
